In [1]:
import seaborn as sns
import re
import matplotlib.pyplot as plt
from datetime import datetime
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None 
%matplotlib inline

In [2]:
survey = pd.read_excel('data/Master.xlsx',"Master")

# Strip spaces from beginning and end of column names, strings, standardize "No Response" as NaN

In [3]:
survey.columns = survey.columns.str.strip()

for col in survey.columns:
    if survey[col].dtype == 'O':
        survey[col] = survey[col].str.strip()
        survey[col] = survey[col].str.replace('_',' ')

survey = survey.replace('NR', np.NaN)
survey = survey.replace('Invalid', np.NaN)

In [4]:
survey = survey.rename(columns={"What is your gender?": "Gender", 
                                "What is your annual household income?": "Income", 
                                "What year were you born?": "Age",
                                "What is the zip code (i.e. postcal code) of the place you currently live in?": "Zipcode", 
                                "What is your employment status?": "Employment status", 
                                "What is your ethnicity?": "Ethnicity",
                                "What industry are you part of?": "Industry",
                                "What country do you currently live in?": "Country"})

# Standardize first letter capitalization

In [5]:
survey['Employment status'] = survey['Employment status'].str.capitalize() 
survey['Gender'] = survey['Gender'].str.capitalize()
survey['Ethnicity'] = survey['Ethnicity'].str.capitalize()
survey['Industry'] = survey['Industry'].str.capitalize()

# Age and Age Group variables

In [6]:
survey['Age'] = 2020 - survey['Age']
survey['Age group'] = pd.cut(x=survey['Age'], bins=[0, 24, 39, 54, 70, 100]).apply(str)

# Create Dummy Variables, (Female, Male, Student, Retired)

In [7]:
survey[['Female','Male']] = pd.get_dummies(survey['Gender'])[['Female','Male']]
survey[['Student','Retired']] = pd.get_dummies(survey['Employment status'])[['Student','Retired']]

# WFH? column cleaning

In [8]:
survey['In the past 3 days, did you work from home (WFH)?'] = survey['In the past 3 days, did you work from home (WFH)?'].str.replace('home on some days','home for some days')
survey['In the past 3 days, did you work from home (WFH)?'] = survey['In the past 3 days, did you work from home (WFH)?'].str.replace('provide an option for me to work from home','provide an option to WFH')

# Income column cleaning

In [9]:
survey['Income'] = survey['Income'].str.replace('Between $150,000 and $199,000','More than $150,000')
survey['Income'] = survey['Income'].str.replace('More than $200,000','More than $150,000')
survey['Income'] = survey['Income'].str.replace('high iii','More than $150,000')
survey['Income'] = survey['Income'].str.replace('high ii','Between $125,000 and $149,999')
survey['Income'] = survey['Income'].str.replace('high i','Between $100,000 and $124,999')
survey['Income'] = survey['Income'].str.replace('middle ii','Between $75,000 and $99,999')
survey['Income'] = survey['Income'].str.replace('middle i','Between $50,000 and $74,999')
survey['Income'] = survey['Income'].str.replace('lower ii','Between $25,000 and $49,999')
survey['Income'] = survey['Income'].str.replace('lower i','Less than $25,000')
survey['Income'] = survey['Income'].str.replace('prefer not to say','Prefer not to say')
survey['Income'] = survey['Income'].str.replace('$','$\$$')

# Ethnicity column cleaning

In [10]:
survey['Ethnicity'] = survey['Ethnicity'].str.replace('White','Caucasian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Hispanic/latinx','Hispanic')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Latino','Hispanic')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Black/african american','Black')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, asian/asian american','Multiracial')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, hispanic/latinx','Multiracial')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Asian/asian american','Asian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, armenian','Multiracial')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Asian, other','Asian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, jewish','Caucasian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, asian','Multiracial')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Multiracial/asian american','Multiracial')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Arab','Other')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Asian, pacific islander','Asian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Prefer_not_to_say','Prefer not to say')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Caucasian, other is how i prefer to identify','Other')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Indian american','Asian')
survey['Ethnicity'] = survey['Ethnicity'].str.replace('Jewish','Other')

# Separate out different versions

In [11]:
pollfish1 = survey[survey['Source']=='Pollfish1']
pollfish2 = survey[survey['Source']=='Pollfish2']
google1 = survey[survey['Source']=='Google1']
google2 = survey[survey['Source']=='Google2']

# Date time column

In [12]:
pollfish1['TimeFinished'] = pd.to_datetime(pollfish1['TimeFinished'], format='%Y-%m-%d %H:%M:%S')
pollfish2['TimeFinished'] = pd.to_datetime(pollfish2['TimeFinished'], format='%Y-%m-%d %H:%M:%S')
google1['TimeFinished'] = pd.to_datetime(google1['TimeFinished'], format='%-m/%d/%Y %-I:%M:%S %p')
google2['TimeFinished'] = pd.to_datetime(google2['TimeFinished'], format='%-m/%d/%Y %-I:%M:%S %p')

In [13]:
google1['TimeFinished'] = pd.to_datetime(google1["TimeFinished"].dt.strftime('%Y-%m-%d %H:%M:%S'))
google2['TimeFinished'] = pd.to_datetime(google2["TimeFinished"].dt.strftime('%Y-%m-%d %H:%M:%S'))

# Mapping of scale questions

In [14]:
google1['On a scale of 1 to 10, how reasonable do you think the public reaction is to COVID-19 now? (1 is under-reacting and 10 is overreacting)'] = google1['On a scale of 1 to 10, how reasonable do you think the public reaction is to COVID-19 now? (1 is under-reacting and 10 is overreacting)'].map({'Strongly disagree': '1', 'Disagree': '3', 'Neutral': '5', 'Agree': '7', 'Strongly agree': '9'}) 
pollfish1['On a scale of 1 to 10, how reasonable do you think the public reaction is to COVID-19 now? (1 is under-reacting and 10 is overreacting)'] = pollfish1['On a scale of 1 to 10, how reasonable do you think the public reaction is to COVID-19 now? (1 is under-reacting and 10 is overreacting)'].map({'Strongly disagree': '1', 'Disagree': '3', 'Neutral': '5', 'Agree': '7', 'Strongly agree': '9'}) 


In [15]:
google1['On a scale of 1 to 10, how much do you trust the government to respond to COVID-19 effectively? (1 is strongly distrust and 10 is strongly trust)'] = google1['On a scale of 1 to 10, how much do you trust the government to respond to COVID-19 effectively? (1 is strongly distrust and 10 is strongly trust)'].map({'Strongly disagree': '1', 'Disagree': '3', 'Neutral': '5', 'Agree': '7', 'Strongly agree': '9'}) 
pollfish1['On a scale of 1 to 10, how much do you trust the government to respond to COVID-19 effectively? (1 is strongly distrust and 10 is strongly trust)'] = pollfish1['On a scale of 1 to 10, how much do you trust the government to respond to COVID-19 effectively? (1 is strongly distrust and 10 is strongly trust)'].map({'Strongly disagree': '1', 'Disagree': '3', 'Neutral': '5', 'Agree': '7', 'Strongly agree': '9'}) 


# Combine all versions

In [16]:
survey = pd.concat([pollfish1, pollfish2, google1, google2], ignore_index=True)

# Checkbox questions

In [17]:
survey = survey.rename(
    columns={'Unnamed: 10': 'In the past 48 hours, I ' + survey['Unnamed: 10'].dropna().unique()[0].lower(),
             'Unnamed: 11': 'In the past 48 hours, I ' + survey['Unnamed: 11'].dropna().unique()[0].lower(),
             'Unnamed: 12': 'In the past 48 hours, I ' + survey['Unnamed: 12'].dropna().unique()[0].lower(),
             'Unnamed: 13': 'In the past 48 hours, I ' + survey['Unnamed: 13'].dropna().unique()[0].lower(),
             'Unnamed: 14': 'In the past 48 hours, I ' + survey['Unnamed: 14'].dropna().unique()[0].lower(),
             'Unnamed: 15': 'In the past 48 hours, I ' + survey['Unnamed: 15'].dropna().unique()[0].lower(),
             'Unnamed: 16': 'In the past 48 hours, I ' + survey['Unnamed: 16'].dropna().unique()[0].lower(),
             'Unnamed: 17': 'In the past 48 hours, I ' + survey['Unnamed: 17'].dropna().unique()[0].lower(),
             'Unnamed: 18': 'In the past 48 hours, I ' + survey['Unnamed: 18'].dropna().unique()[0].lower(),
             'Unnamed: 19': 'In the past 48 hours, I ' + survey['Unnamed: 19'].dropna().unique()[0].lower(),
             'Unnamed: 21': 
             'Is any of your loved ones at higher risk of COVID-19? ' + survey['Unnamed: 21'].dropna().unique()[0],
             'Unnamed: 22': 
             'Is any of your loved ones at higher risk of COVID-19? ' + survey['Unnamed: 22'].dropna().unique()[0],
             'Unnamed: 23': 
             'Is any of your loved ones at higher risk of COVID-19? ' + survey['Unnamed: 23'].dropna().unique()[0],
             'Unnamed: 24': 
             'Is any of your loved ones at higher risk of COVID-19? ' + survey['Unnamed: 24'].dropna().unique()[0]
})
    

In [18]:
survey = survey.drop(['In the past 48 hours, I... (check all that apply)', 
                      'Is any of your loved ones at higher risk of COVID-19? (check all that apply)'], axis=1)

In [19]:
survey[[col for col in survey.columns if 'In the past 48 hours' in col]] = survey[[col for col in survey.columns if 'In the past 48 hours' in col]].fillna('False')
survey[[col for col in survey.columns if 'Is any of your loved ones at higher risk of COVID-19?' in col]] = survey[[col for col in survey.columns if 'Is any of your loved ones at higher risk of COVID-19?' in col]].fillna('False')


In [20]:
for col in [col for col in survey.columns if 'In the past 48 hours' in col]:
    survey.loc[~survey[col].str.contains('False'), col] = 'True'
    
for col in [col for col in survey.columns if 'Is any of your loved ones at higher risk of COVID-19?' in col]:
    survey.loc[~survey[col].str.contains('False'), col] = 'True'